In [ ]:
# !pip install plotly==4.8.1

In [1]:
import pandas as pd
from datetime import datetime as dt
import time
import plotly.express as px

# Loading User Meta Data

In [2]:
user = pd.read_csv("assets/data/wp_usermeta.csv")

In [3]:
user = user.loc[user["meta_key"] == "length"]
user

,umeta_id,user_id,meta_key,meta_value
47,194,10,length,75


# Loading Appointment Data

In [4]:
ap = pd.read_csv("assets/data/wp_jet_appointments.csv")
ap

,ID,status,service,provider,user_email,date,slot,slot_end,order_id,user_id,phone,comments
0,1,pending,404,415,abrahammrjones@gmail.com,1591833600,1591876800,1591880400,NaN,NaN,17734206926,Please work
1,2,pending,456,417,abrahammrjones@gmail.com,1591833600,1591880400,1591891200,NaN,10.0,17734206926,Book me up
2,3,pending,458,415,abrahammrjones@gmail.com,1591833600,1591884000,1591891200,NaN,10.0,17734206926,NaN
3,4,pending,460,415,abrahammrjones@gmail.com,1591833600,1591880400,1591884000,NaN,10.0,17734206926,NaN
4,5,pending,442,415,abrahammrjones@gmail.com,1591833600,1591902000,1591905600,NaN,10.0,17734206926,NaN


### Convert data series to DateTime format

In [5]:
# Using a unix epoch time
ap["date"] = pd.to_datetime(ap["date"], unit='s')
ap["slot"] = pd.to_datetime(ap["slot"], unit='s').dt.time
ap["slot_end"] = pd.to_datetime(ap["slot_end"], unit='s').dt.time
ap

,ID,status,service,provider,user_email,date,slot,slot_end,order_id,user_id,phone,comments
0,1,pending,404,415,abrahammrjones@gmail.com,2020-06-11,12:00:00,13:00:00,NaN,NaN,17734206926,Please work
1,2,pending,456,417,abrahammrjones@gmail.com,2020-06-11,13:00:00,16:00:00,NaN,10.0,17734206926,Book me up
2,3,pending,458,415,abrahammrjones@gmail.com,2020-06-11,14:00:00,16:00:00,NaN,10.0,17734206926,NaN
3,4,pending,460,415,abrahammrjones@gmail.com,2020-06-11,13:00:00,14:00:00,NaN,10.0,17734206926,NaN
4,5,pending,442,415,abrahammrjones@gmail.com,2020-06-11,19:00:00,20:00:00,NaN,10.0,17734206926,NaN


### Converting 24 hours to 12 hours AM/PM

In [6]:
ap["slot"] = ap["slot"].apply(lambda x: x.strftime('%I:%M %p'))
ap["slot_end"] = ap["slot_end"].apply(lambda x: x.strftime('%I:%M %p'))
ap

,ID,status,service,provider,user_email,date,slot,slot_end,order_id,user_id,phone,comments
0,1,pending,404,415,abrahammrjones@gmail.com,2020-06-11,12:00 PM,01:00 PM,NaN,NaN,17734206926,Please work
1,2,pending,456,417,abrahammrjones@gmail.com,2020-06-11,01:00 PM,04:00 PM,NaN,10.0,17734206926,Book me up
2,3,pending,458,415,abrahammrjones@gmail.com,2020-06-11,02:00 PM,04:00 PM,NaN,10.0,17734206926,NaN
3,4,pending,460,415,abrahammrjones@gmail.com,2020-06-11,01:00 PM,02:00 PM,NaN,10.0,17734206926,NaN
4,5,pending,442,415,abrahammrjones@gmail.com,2020-06-11,07:00 PM,08:00 PM,NaN,10.0,17734206926,NaN


### Replacing provider ID with name

In [7]:
ap["provider"] = ap["provider"].replace(415, "City Winery at the Chicago Riverwalk")
ap["provider"] = ap["provider"].replace(417, "Marina City")
ap["provider"] = ap["provider"].replace(421, "Pizzeria Portofino")

### Creating dataframe for the provider dockage length

In [8]:
pro_doc = {
                "provider" : ["City Winery at the Chicago Riverwalk", "Marina City", "Pizzeria Portofino"],
                "dockage" : [210, 55, 120]
            }
pro_doc = pd.DataFrame(pro_doc)
pro_doc

,provider,dockage
0,City Winery at the Chicago Riverwalk,210
1,Marina City,55
2,Pizzeria Portofino,120


### Replacing service id with name

In [9]:
ap["service"] = ap["service"].replace(478, "Dockage Under 24' 1hr")
ap["service"] = ap["service"].replace(476, "Dockage Under 24' 2hr")
ap["service"] = ap["service"].replace(474, "Dockage Under 24' 3hr")
ap["service"] = ap["service"].replace(472, "Dockage 25' to 34' 1hr")
ap["service"] = ap["service"].replace(470, "Dockage 25' to 34' 2hr")
ap["service"] = ap["service"].replace(468, "Dockage 25' to 34' 3hr")
ap["service"] = ap["service"].replace(466, "Dockage 35' to 44' 1hr")
ap["service"] = ap["service"].replace(464, "Dockage 35' to 44' 2hr")
ap["service"] = ap["service"].replace(462, "Dockage 35' to 44' 3hr")
ap["service"] = ap["service"].replace(460, "Dockage 45' to 54' 1hr")
ap["service"] = ap["service"].replace(458, "Dockage 45' to 54' 2hr")
ap["service"] = ap["service"].replace(456, "Dockage 45' to 54' 3hr")
ap["service"] = ap["service"].replace(454, "Dockage 55' to 64' 1hr")
ap["service"] = ap["service"].replace(452, "Dockage 55' to 64' 2hr")
ap["service"] = ap["service"].replace(450, "Dockage 55' to 64' 3hr")
ap["service"] = ap["service"].replace(448, "Dockage 65' to 124' 1hr")
ap["service"] = ap["service"].replace(446, "Dockage 65' to 124' 2hr")
ap["service"] = ap["service"].replace(444, "Dockage 65' to 124' 3hr")
ap["service"] = ap["service"].replace(442, "Dockage 125' and more 1hr")
ap["service"] = ap["service"].replace(440, "Dockage 125' and more 2hr")
ap["service"] = ap["service"].replace(438, "Dockage 125' and more 3hr")
ap

,ID,status,service,provider,user_email,date,slot,slot_end,order_id,user_id,phone,comments
0,1,pending,404,City Winery at the Chicago Riverwalk,abrahammrjones@gmail.com,2020-06-11,12:00 PM,01:00 PM,NaN,NaN,17734206926,Please work
1,2,pending,Dockage 45' to 54' 3hr,Marina City,abrahammrjones@gmail.com,2020-06-11,01:00 PM,04:00 PM,NaN,10.0,17734206926,Book me up
2,3,pending,Dockage 45' to 54' 2hr,City Winery at the Chicago Riverwalk,abrahammrjones@gmail.com,2020-06-11,02:00 PM,04:00 PM,NaN,10.0,17734206926,NaN
3,4,pending,Dockage 45' to 54' 1hr,City Winery at the Chicago Riverwalk,abrahammrjones@gmail.com,2020-06-11,01:00 PM,02:00 PM,NaN,10.0,17734206926,NaN
4,5,pending,Dockage 125' and more 1hr,City Winery at the Chicago Riverwalk,abrahammrjones@gmail.com,2020-06-11,07:00 PM,08:00 PM,NaN,10.0,17734206926,NaN


# Merging appointment and user dataframes

In [10]:
ap_m = pd.merge(ap, user, how="inner", on="user_id")
ap_m

,ID,status,service,provider,user_email,date,slot,slot_end,order_id,user_id,phone,comments,umeta_id,meta_key,meta_value
0,2,pending,Dockage 45' to 54' 3hr,Marina City,abrahammrjones@gmail.com,2020-06-11,01:00 PM,04:00 PM,NaN,10.0,17734206926,Book me up,194,length,75
1,3,pending,Dockage 45' to 54' 2hr,City Winery at the Chicago Riverwalk,abrahammrjones@gmail.com,2020-06-11,02:00 PM,04:00 PM,NaN,10.0,17734206926,NaN,194,length,75
2,4,pending,Dockage 45' to 54' 1hr,City Winery at the Chicago Riverwalk,abrahammrjones@gmail.com,2020-06-11,01:00 PM,02:00 PM,NaN,10.0,17734206926,NaN,194,length,75
3,5,pending,Dockage 125' and more 1hr,City Winery at the Chicago Riverwalk,abrahammrjones@gmail.com,2020-06-11,07:00 PM,08:00 PM,NaN,10.0,17734206926,NaN,194,length,75


# Merging User Appointment and Provider Dockage dataframes

In [11]:
dock_data = pd.merge(pro_doc, ap_m, how="inner", on="provider")
dock_data

,provider,dockage,ID,status,service,user_email,date,slot,slot_end,order_id,user_id,phone,comments,umeta_id,meta_key,meta_value
0,City Winery at the Chicago Riverwalk,210,3,pending,Dockage 45' to 54' 2hr,abrahammrjones@gmail.com,2020-06-11,02:00 PM,04:00 PM,NaN,10.0,17734206926,NaN,194,length,75
1,City Winery at the Chicago Riverwalk,210,4,pending,Dockage 45' to 54' 1hr,abrahammrjones@gmail.com,2020-06-11,01:00 PM,02:00 PM,NaN,10.0,17734206926,NaN,194,length,75
2,City Winery at the Chicago Riverwalk,210,5,pending,Dockage 125' and more 1hr,abrahammrjones@gmail.com,2020-06-11,07:00 PM,08:00 PM,NaN,10.0,17734206926,NaN,194,length,75
3,Marina City,55,2,pending,Dockage 45' to 54' 3hr,abrahammrjones@gmail.com,2020-06-11,01:00 PM,04:00 PM,NaN,10.0,17734206926,Book me up,194,length,75


In [12]:
# dock_data.to_csv("assets/data/dockage.csv", index=False)

In [13]:
dock_data = dock_data[["provider", "dockage", "date", "slot", "slot_end", "meta_value"]]
dock_data = dock_data.rename(columns={"slot":"start_time", "slot_end": "end_time","meta_value": "boat_length"})
cw = dock_data.loc[dock_data["provider"] == "City Winery at the Chicago Riverwalk"]
cw

,provider,dockage,date,start_time,end_time,boat_length
0,City Winery at the Chicago Riverwalk,210,2020-06-11,02:00 PM,04:00 PM,75
1,City Winery at the Chicago Riverwalk,210,2020-06-11,01:00 PM,02:00 PM,75
2,City Winery at the Chicago Riverwalk,210,2020-06-11,07:00 PM,08:00 PM,75


In [ ]:
# if ("02:00 PM" >= cw["start_time"] & "02:00 PM" < cw["end_time"]):
#     cw["02:00 PM"] = cw["boat_length"]
# cw

In [ ]:
# cw.to_csv("assets/data/cw.csv", index=False)

In [ ]:
# cw["boat_length"] = cw["boat_length"].astype('int64')

In [ ]:
data = px.data.gapminder()

data_canada = data[data.country == 'Canada']
fig = px.bar(data_canada, x='year', y='pop',
             hover_data=['lifeExp', 'gdpPercap'], color='lifeExp',
             labels={'pop':'population of Canada'}, height=400)
fig.show()